In [2]:
import geopandas as gpd
import pickle as pkl
import json
import pandas as pd
from shapely.geometry import Point, shape
from shapely.geometry.polygon import Polygon

from sqlalchemy import create_engine, text
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker

In [5]:
%load_ext sql

In [6]:
%sql postgresql://teleport@localhost:2024/perchwell


In [7]:
def pull_data(sql_query:str, file_out="pw.csv", port_num=2024):
    """
    For a given sql_query, this function will connect to Perchwell's database, 
    pull the corresponding data, save it to a csv in the file specified 
    (default is pw.csv), and returns that data in the form of a Pandas
    DataFrame.

    Args:
        sql_query (str): query whose results you wish to download
        file_out (str, optional): filepath for the output file. Defaults to 
            "pw.csv".
        port_num (int, optional): Port number for Teleport Connection. Defaults
            to 2023.

    Returns:
        pd.DataFrame: DataFrame containing results of sql_query 
    """    
    pw_df = []
    prev_row_count = -1
    current_row = 0
    # Create the SQLAlchemy engine
    engine = create_engine(f"postgresql://teleport:@localhost:{port_num}/perchwell")

    # in order to get around the 300k row limit
    while prev_row_count <= current_row:
        current_row = len(pw_df)
        if prev_row_count == current_row:
            print(f"final row count: {current_row}")
            break
        # Define the SQL query string, but keep last line
        query_string = f"""{sql_query}
            -- NOTE: DO NOT DELETE BELOW THIS
            LIMIT 100000 OFFSET {current_row};
            """
        try:
            # Execute the query
            with engine.connect() as conn:
                result = conn.execute(text(query_string))

                # Process the query result
                for row in result:
                    # Access the row data
                    pw_df.append(row)
        except:
            # in case of timeout
            engine = create_engine(f"postgresql://teleport:@localhost:{port_num}/perchwell")
            with engine.connect() as conn:
                result = conn.execute(text(query_string))
                for row in result:
                    pw_df.append(row)
        print(f"{len(pw_df)} rows appended")
        prev_row_count = current_row
    pw_df = pd.DataFrame(pw_df)
    #pw_df.to_csv(file_out)
    return pw_df

In [ ]:
try:
    with open('data/buildings_final.pkl', 'rb') as f:
        preso = pkl.load(f)
    print(preso.shape)
except FileNotFoundError:
    query_string = f'''select b.*, st_asgeojson(b.geometry) as geometry_json
            from buildings b
            ---join with geographies
            inner join (
                select b_geo.building_id, geo.geometry, geo.name
                from building_geographies b_geo
                inner join geographies geo
                    on b_geo.geography_id = geo.id
                where geo.name in ('Manhattan', 'Brooklyn', 'Bronx', 'Queens')
                ) bg on b.id = bg.building_id
            left join (
                select
                    building_id
                    , count(*) as num_other_listings
                    , max(created_at) as last_listing
                from listings
                where not(
                        lower(source) like '%mlsli%' 
                        or lower(source) like '%rls%' 
                        or source = 'real_plus')
                group by building_id
            ) l on l.building_id = b.id
            left join (
                select
                   building_id
                    , count(distinct(id)) as num_rls
                    , max(created_at) as last_rls_listing
                from listings
                where --not(lower(source) like '%mlsli%')
                  lower(source) like '%rls%' or source = 'real_plus'
                group by building_id
            ) rls on rls.building_id = b.id
            left join (
                select
                    building_id
                    , count(distinct(id)) as num_mlsli
                    , max(created_at) as last_mlsli_listing
                from listings
                where lower(source) like '%mlsli%'
                group by building_id
            ) mlsli on mlsli.building_id = b.id
            left join (
                select 
                    building_id
                    , count(distinct(acris_document_id)) as num_acris
                    , max(created_at) as last_acris_created
                    , max(sale_date) as last_acris_sale
                from historicals
                group by building_id
                order by count(distinct(acris_document_id)) desc
            ) a on b.id = a.building_id
            '''
    preso = pull_data(query_string)
    with open('data/buildings_final.pkl', 'wb') as f:
        pkl.dump(preso, f)

In [28]:
preso = preso[preso.geometry_json.notnull()]
# Transform normal df to geopandas df
gdf = gpd.GeoDataFrame(preso)
# Transform geojson to shapes
for i,r in gdf.iterrows():
    gdf.at[i, 'geometry_json'] = shape(json.loads(gdf['geometry_json'][i]))
    
# Set geometry for geodf
gdf = gdf.set_geometry('geometry_json')



In [ ]:
PLdf = pd.read_csv('/Users/perchwellallusers/Documents/GitHub/data-quality/Data_Refresh/PLUTO/PLUTO_Only_df.csv')

In [ ]:
sp2 = gpd.sjoin(PLgdf, gdf, how = 'inner', op = 'within')